In [1]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark import SparkConf,SparkContext
%matplotlib inline
import numpy as np
import subprocess
DATA_PATH = "../mlb_data/games.csv"

In [2]:
#Read csv file to dataframe
#=====your code here==========
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
# income_df = spark.read.format("com.databricks.spark.csv").options(header="false", inferschema="true").load(DATA_PATH)
data = spark.read.options(header = "false", inferSchema = "True").csv(DATA_PATH)

/home/whitechubbyguy/myenv/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
from functools import reduce
#change the column names of dataframe
df = data.withColumnRenamed('_c0','Game').withColumnRenamed('_c1','away').withColumnRenamed('_c2','away-record')\
.withColumnRenamed('_c3', 'awayaway-record').withColumnRenamed('_c4', 'home')\
.withColumnRenamed('_c5','home-record').withColumnRenamed('_c6', 'homehome-record').withColumnRenamed('_c7', 'away-score')\
.withColumnRenamed('_c8', 'home-score').withColumnRenamed('_c9', 'postseason info').withColumnRenamed('_c10', 'Walks Issued - Away')\
.withColumnRenamed('_c11', 'Walks Issued - Home').withColumnRenamed('_c12','Stolen Bases - Away')\
.withColumnRenamed('_c13', 'Stolen Bases - Home').withColumnRenamed('_c14', 'Strikeouts Thrown - Away').withColumnRenamed('_c15', 'Strikeouts Thrown - Home')\
.withColumnRenamed('_c16', 'Total Bases - Away').withColumnRenamed('_c17', 'Total Bases - Home').withColumnRenamed('_c18', 'Stadium')\
.withColumnRenamed('_c19', 'Date').withColumnRenamed('_c20', 'Location').withColumnRenamed('_c21', 'Odds')\
.withColumnRenamed('_c22', 'O/U').withColumnRenamed('_c23', 'Attendance').withColumnRenamed('_c24', 'Capacity')\
.withColumnRenamed('_c25', 'Duration').withColumnRenamed('_c26', 'Umpires').withColumnRenamed('_c27', 'WIN - Pitcher - Stats')\
.withColumnRenamed('_c28', 'WIN - Pitcher - Id').withColumnRenamed('_c29', 'WIN - Pitcher - Name').withColumnRenamed('_c30', 'WIN - Pitcher - AbbrName')\
.withColumnRenamed('_c31', 'WIN - Pitcher - Record').withColumnRenamed('_c32', 'LOSS - Pitcher - Stats').withColumnRenamed('_c33', 'LOSS - Pitcher - Id')\
.withColumnRenamed('_c34', 'LOSS - Pitcher - Name').withColumnRenamed('_c35', 'LOSS - Pitcher - AbbrName').withColumnRenamed('_c36', 'LOSS - Pitcher - Record')\
.withColumnRenamed('_c37', 'SAVE - Pitcher - Stats').withColumnRenamed('_c38', 'SAVE - Pitcher - Id').withColumnRenamed('_c39', 'SAVE - Pitcher - Name')\
.withColumnRenamed('_c40', 'SAVE - Pitcher - AbbrName').withColumnRenamed('_c41', 'SAVE - Pitcher - Record').withColumnRenamed('_c42', 'Extra Innings')
df.printSchema()
df.show(2)

dataset = df

root
 |-- Game: string (nullable = true)
 |-- away: string (nullable = true)
 |-- away-record: string (nullable = true)
 |-- awayaway-record: string (nullable = true)
 |-- home: string (nullable = true)
 |-- home-record: string (nullable = true)
 |-- homehome-record: string (nullable = true)
 |-- away-score: string (nullable = true)
 |-- home-score: string (nullable = true)
 |-- postseason info: string (nullable = true)
 |-- Walks Issued - Away: string (nullable = true)
 |-- Walks Issued - Home: string (nullable = true)
 |-- Stolen Bases - Away: string (nullable = true)
 |-- Stolen Bases - Home: string (nullable = true)
 |-- Strikeouts Thrown - Away: string (nullable = true)
 |-- Strikeouts Thrown - Home: string (nullable = true)
 |-- Total Bases - Away: string (nullable = true)
 |-- Total Bases - Home: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Odds: string (nullable = true)
 |-- O/U:

In [4]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit, col
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql.types import IntegerType,BooleanType,DateType,StringType

In [5]:
# filter data for away
mlb_teams = ["PIT", "TB", "KC", 'MIL', "BAL", "TEX", 'ATL', 'CIN', 'SD', 'ARI', 'LAA', 'OAK', 'NYY', 'SEA', 'WSH', \
             'TOR', 'NYM', 'SF', 'MIN', 'PHI', 'HOU', 'CHW', 'BOS', 'STL', 'DET', 'LAD', 'COL', 'CHC', 'CLE', 'MIA']
rdd = df.rdd.map(lambda x: (x[1], x[7], x[10], x[12], x[14], x[16], x[19], x[4], x[8], x[11], x[13], x[15], x[17]))
df2=rdd.toDF(["team","score","walksissued","stolenbases","strikeoutsthrown", "totalbases", 'date', 'opponent', 'op_score',"op_walksissued","op_stolenbases","op_strikeoutsthrown", "op_totalbases"])

df2 = df2.na.drop().filter(df2.team.isin(mlb_teams)==True)
df2 = df2.withColumn("home", lit(0))
df2.show()
df2.count()

+----+-----+-----------+-----------+----------------+----------+-----------------+--------+--------+--------------+--------------+-------------------+-------------+----+
|team|score|walksissued|stolenbases|strikeoutsthrown|totalbases|             date|opponent|op_score|op_walksissued|op_stolenbases|op_strikeoutsthrown|op_totalbases|home|
+----+-----+-----------+-----------+----------------+----------+-----------------+--------+--------+--------------+--------------+-------------------+-------------+----+
| STL|  1.0|        5.0|        0.0|             5.0|       5.0|2016-04-03T17:00Z|     PIT|     4.0|           5.0|           0.0|               14.0|         13.0|   0|
| TOR|  5.0|        1.0|        0.0|             7.0|      11.0|2016-04-03T20:00Z|      TB|     3.0|           3.0|           0.0|               16.0|         11.0|   0|
| NYM|  3.0|        2.0|        0.0|             3.0|       8.0|2016-04-04T00:30Z|      KC|     4.0|           6.0|           1.0|                9.0|

13429

In [8]:
df_final = df2.withColumn("team",col("team").cast(StringType())) \
    .withColumn("score",col("score").cast(IntegerType())) \
    .withColumn("walksissued",col("walksissued").cast(IntegerType())) \
    .withColumn("stolenbases",col("stolenbases").cast(IntegerType())) \
    .withColumn("strikeoutsthrown",col("strikeoutsthrown").cast(IntegerType())) \
    .withColumn("totalbases",col("totalbases").cast(IntegerType())) \
    .withColumn("home",col("home").cast(IntegerType())) \
    .withColumn("op_score",col("op_score").cast(IntegerType())) \
    .withColumn("op_walksissued",col("op_walksissued").cast(IntegerType())) \
    .withColumn("op_stolenbases",col("op_stolenbases").cast(IntegerType())) \
    .withColumn("op_strikeoutsthrown",col("op_strikeoutsthrown").cast(IntegerType())) \
    .withColumn("op_totalbases",col("op_totalbases").cast(IntegerType())) \
    .withColumn("opponent",col("opponent").cast(StringType())) \
    .withColumn("date",col("date").cast(DateType()))
df_final.printSchema()
df_final.show(10)
print(df_final.count())
df_final.write.option("header",True).csv("../data_processed/pyspark_data.csv")

root
 |-- team: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- walksissued: integer (nullable = true)
 |-- stolenbases: integer (nullable = true)
 |-- strikeoutsthrown: integer (nullable = true)
 |-- totalbases: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- opponent: string (nullable = true)
 |-- op_score: integer (nullable = true)
 |-- op_walksissued: integer (nullable = true)
 |-- op_stolenbases: integer (nullable = true)
 |-- op_strikeoutsthrown: integer (nullable = true)
 |-- op_totalbases: integer (nullable = true)
 |-- home: integer (nullable = false)

+----+-----+-----------+-----------+----------------+----------+----------+--------+--------+--------------+--------------+-------------------+-------------+----+
|team|score|walksissued|stolenbases|strikeoutsthrown|totalbases|      date|opponent|op_score|op_walksissued|op_stolenbases|op_strikeoutsthrown|op_totalbases|home|
+----+-----+-----------+-----------+----------------+----------+-----

In [9]:
from google.cloud import bigquery
bucket = "6893_data_cc4805"
output_directory_pyspark = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/pyspark'.format(bucket)
lsCommand = ['gsutil', 'cp', '../data_processed/pyspark_data.csv', output_directory_pyspark]
subprocess.check_output(lsCommand)

b''

In [22]:
client = bigquery.Client()
table_id = "pyspark_dataset.pyspark"
# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("team", "STRING"),
        bigquery.SchemaField("score", "INTEGER"),
        bigquery.SchemaField("walksissued", "INTEGER"),
        bigquery.SchemaField("stolenbases", "INTEGER"),
        bigquery.SchemaField("strikeoutsthrown", "INTEGER"),
        bigquery.SchemaField("totalbases", "INTEGER"),
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("opponent", "STRING"),
        bigquery.SchemaField("op_score", "INTEGER"),
        bigquery.SchemaField("op_walksissued", "INTEGER"),
        bigquery.SchemaField("op_stolenbases", "STRING"),
        bigquery.SchemaField("op_strikeoutsthrown", "INTEGER"),
        bigquery.SchemaField("op_totalbases", "INTEGER"),
        bigquery.SchemaField("home", "INTEGER"),
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/pyspark'.format(bucket)

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 13429 rows.
